In [1]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 
from keras.models import model_from_json
from keras.preprocessing.image import ImageDataGenerator
from skimage.transform import resize

Using TensorFlow backend.


In [2]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']
for i in test_dicoms:
    ds = pydicom.dcmread(i)
    print(ds.StudyDescription, ds.Modality, ds.PatientPosition, ds.BodyPartExamined)

No Finding DX PA CHEST
Cardiomegaly DX AP CHEST
Effusion DX AP CHEST
No Finding DX PA RIBCAGE
No Finding CT PA CHEST
No Finding DX XX CHEST


In [3]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    # todo
    
    ds = pydicom.dcmread(filename)       

    if ds.Modality == 'DX' and ds.PatientPosition in ['AP', 'PA'] and ds.BodyPartExamined in ['CHEST']:
        img = ds.pixel_array
        return img
    else:
        print(f'{filename} not suitable')
        return None
    return
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_size): 
    # todo

    normal_img  = (img -np.mean(img))/ np.std(img)
    res_img = resize(normal_img, (img_size[1], img_size[2]))
    proc_img = res_img.reshape((1, img_size[1], img_size[2], 1))
    proc_img = np.repeat(proc_img, img_size[3], axis=3)
    return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    # todo
    json_file = open(model_path, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    
    model = model_from_json(loaded_model_json)
    # load weights into new model
    model.load_weights(weight_path)
    print("Loaded model from disk")
    
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    # todo    
    preds=model.predict(img)
    if preds>thresh:
        prediction=1
    else:
        prediction=0
    return prediction 

In [4]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = "my_model.json"#path to saved model .json
weight_path = "best_my_model.best.hdf5"#path to saved best weights

IMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16

my_model = load_model(model_path, weight_path)#loads model
thresh = 0.48#loads the threshold they chose for model classification 

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = check_dicom(i)
    
    if img is None:
        continue
        
    img_proc = preprocess_image(img,IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
    if pred==1:
        print("Image {} has Pneumonia".format(i))
    else:
        print("Image {} does not have pneumonia".format(i))

Loaded model from disk
Image test1.dcm does not have pneumonia
Image test2.dcm does not have pneumonia
Image test3.dcm does not have pneumonia
test4.dcm not suitable
test5.dcm not suitable
test6.dcm not suitable
